In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
import glob

temp = pd.read_csv('data/Entire_Season.csv')

In [2]:
data = '2023-11-19'

In [3]:
def verify_over(caminho):
    df = pd.read_csv(caminho)
    
    resultados = pd.merge(df, temp[['Date', 'Home', 'Away', 'Home_Pts', 'Away_Pts']], on=['Date', 'Home', 'Away'], how='left')

    resultados.loc[((resultados['Home_Pts'] + resultados['Away_Pts']) > resultados['Over_Line']), 'R'] = 'green'
    resultados.loc[(resultados['R'] != 'green'), 'R'] = 'red'

    resultados = resultados[['Home', 'Away', 'Home_Pts', 'Away_Pts', 'Over_Line', 'Date', 'Odds_Over', 'R']]

    return resultados


def verify_back_home(caminho):
    df = pd.read_csv(caminho)
    
    resultados = pd.merge(df, temp[['Date', 'Home', 'Away', 'Home_Pts', 'Away_Pts']], on=['Date', 'Home', 'Away'], how='left')

    resultados.loc[(resultados['Home_Pts'] > resultados['Away_Pts']), 'R'] = 'green'
    resultados.loc[(resultados['R'] != 'green'), 'R'] = 'red'

    resultados = resultados[['Home', 'Away', 'Home_Pts', 'Away_Pts', 'Date', 'Odds_H', 'R']]

    return resultados


def verify_positive_handicap(caminho):
    df = pd.read_csv(caminho)

    resultados = pd.merge(df, temp[['Date', 'Home', 'Away', 'Home_Pts', 'Away_Pts']], on=['Date', 'Home', 'Away'], how='left')
    
    resultados.loc[(resultados['HA_Line']) > 0, 'Zebra'] = 'Casa'
    resultados.loc[(resultados['HA_Line']) < 0, 'Zebra'] = 'Fora'

    # Odds
    resultados.loc[(resultados['Zebra']) == 'Casa', 'Odds'] = resultados['HA_Odds_H']
    resultados.loc[(resultados['Zebra']) == 'Fora', 'Odds'] = resultados['HA_Odds_A']
    

    resultados.loc[((resultados['Zebra'] == 'Casa') & ((resultados['Home_Pts'] + resultados['HA_Line']) < resultados['Away_Pts'])), 'R'] = 'red'
    resultados.loc[(resultados['R'] != 'red'), 'R'] = 'green'
    resultados.loc[((resultados['Zebra'] == 'Fora') & ((resultados['Home_Pts'] + resultados['HA_Line']) > resultados['Away_Pts'])), 'R'] = 'red'

    resultados = resultados[['Home', 'Away', 'Home_Pts', 'Away_Pts', 'HA_Line', 'Date', 'Odds', 'R']]


def verify_negative_handicap(caminho):
    df = pd.read_csv(caminho)

    resultados = pd.merge(df, temp[['Date', 'Home', 'Away', 'Home_Pts', 'Away_Pts']], on=['Date', 'Home', 'Away'], how='left')
    
    resultados.loc[(resultados['HA_Line']) > 0, 'Zebra'] = 'Casa'
    resultados.loc[(resultados['HA_Line']) < 0, 'Zebra'] = 'Fora'

    # Odds
    resultados.loc[(resultados['Zebra']) == 'Casa', 'Odds'] = resultados['HA_Odds_A']
    resultados.loc[(resultados['Zebra']) == 'Fora', 'Odds'] = resultados['HA_Odds_H']
    

    resultados.loc[((resultados['Zebra'] == 'Casa') & ((resultados['Home_Pts'] + resultados['HA_Line']) < resultados['Away_Pts'])), 'R'] = 'green'
    resultados.loc[(resultados['R'] != 'green'), 'R'] = 'red'
    resultados.loc[((resultados['Zebra'] == 'Fora') & ((resultados['Home_Pts'] + resultados['HA_Line']) > resultados['Away_Pts'])), 'R'] = 'green'

    resultados = resultados[['Home', 'Away', 'Home_Pts', 'Away_Pts', 'HA_Odds_H', 'HA_Odds_A', 'HA_Line', 'Date', 'Odds', 'R']]

    return resultados

In [4]:
caminhos = glob.glob(f'apostas_do_dia/{data}/*.csv')
print(f'{len(caminhos)} métodos com apostas para a data informada.\n')

for caminho in caminhos:
    file_name = caminho.split('\\')[1]
    file_name = file_name.split('.')[0]

    if 'over' in caminho:
        df = verify_over(caminho)
        df.to_excel(f'conferencia/{file_name}.xlsx', index=False)
    elif 'back_home' in caminho:
        df = verify_back_home(caminho)
        df.to_excel(f'conferencia/{file_name}.xlsx', index=False)
    elif 'handicap_positivo' in caminho:
        df = verify_positive_handicap(caminho)
        df.to_excel(f'conferencia/{file_name}.xlsx', index=False)
    elif 'handicap_negativo' in caminho:
        df = verify_negative_handicap(caminho)
        df.to_excel(f'conferencia/{file_name}.xlsx', index=False)
    else:
        print(f'Nenhum método encontrado no caminho {caminho}\n')

1 métodos com apostas para a data informada.

